In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [20]:
def generate_time_series(batch_size, n_steps):
  freq1, freq2, offset1, offset2 = np.random.rand(4, batch_size, 1)
  time = np.linspace(0, 1, n_steps)
  series =  0.5 * np.sin((time - offset1) * (freq1 * 10 + 10))
  series +=  0.2 * np.sin((time - offset2) * (freq2 * 10 + 10))
  series +=  0.1 * np.random.rand(batch_size, n_steps) * 0.5
  return series[..., np.newaxis].astype(np.float32)

print(generate_time_series(32, 5).shape)

(32, 5, 1)


In [22]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_val, y_val = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [25]:
# naive forecasting (last element in data as prediction)
# difficult to outperform

print(np.mean(keras.losses.mean_squared_error(y_val, X_val[:, -1])))

0.014027807


In [27]:
# FFNN

model = keras.models.Sequential([
  keras.layers.Flatten(input_shape=[50, 1]),
  keras.layers.Dense(1),
])

model.compile(
  loss=keras.losses.MSE,
  metrics=[keras.metrics.MSE],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train, epochs=20)

print(tf.reduce_mean(keras.losses.MSE(y_test, model.predict(X_test))))

Epoch 1/20
219/219 [==============================] - 0s 389us/step - loss: 0.0472 - mean_squared_error: 0.0472
Epoch 2/20
219/219 [==============================] - 0s 372us/step - loss: 0.0051 - mean_squared_error: 0.0051
Epoch 3/20
219/219 [==============================] - 0s 416us/step - loss: 0.0038 - mean_squared_error: 0.0038
Epoch 4/20
219/219 [==============================] - 0s 407us/step - loss: 0.0032 - mean_squared_error: 0.0032
Epoch 5/20
219/219 [==============================] - 0s 387us/step - loss: 0.0026 - mean_squared_error: 0.0026
Epoch 6/20
219/219 [==============================] - 0s 373us/step - loss: 0.0021 - mean_squared_error: 0.0021
Epoch 7/20
219/219 [==============================] - 0s 362us/step - loss: 0.0017 - mean_squared_error: 0.0017
Epoch 8/20
219/219 [==============================] - 0s 356us/step - loss: 0.0014 - mean_squared_error: 0.0014
Epoch 9/20
219/219 [==============================] - 0s 364us/step - loss: 0.0012 - mean_squared_error:

In [33]:
# Simple RNN

model = keras.models.Sequential([
  keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

model.compile(
  loss=keras.losses.MSE,
  metrics=[keras.metrics.MSE],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train, epochs=20)

print(tf.reduce_mean(keras.losses.MSE(y_test, model.predict(X_test))))

Epoch 1/20
219/219 [==============================] - 1s 2ms/step - loss: 0.1638 - mean_squared_error: 0.1638
Epoch 2/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0966 - mean_squared_error: 0.0966
Epoch 3/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0694 - mean_squared_error: 0.0694
Epoch 4/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0529 - mean_squared_error: 0.0529
Epoch 5/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0425 - mean_squared_error: 0.0425
Epoch 6/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0355 - mean_squared_error: 0.0355
Epoch 7/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0305 - mean_squared_error: 0.0305
Epoch 8/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0267 - mean_squared_error: 0.0267
Epoch 9/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0237 - mean_squared_error: 0.0237
Epoch 10/2

In [37]:
# Deep RNN

model = keras.models.Sequential([
  keras.layers.SimpleRNN(20, input_shape=[None, 1], return_sequences=True),
  keras.layers.SimpleRNN(20, return_sequences=False),
  keras.layers.Dense(1),
])

model.compile(
  loss=keras.losses.MSE,
  metrics=[keras.metrics.MSE],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train, epochs=20)

print(tf.reduce_mean(keras.losses.MSE(y_test, model.predict(X_test))))

Epoch 1/20
219/219 [==============================] - 2s 5ms/step - loss: 0.0329 - mean_squared_error: 0.0329
Epoch 2/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0018 - mean_squared_error: 0.0018
Epoch 3/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0012 - mean_squared_error: 0.0012
Epoch 4/20
219/219 [==============================] - 1s 5ms/step - loss: 9.7958e-04 - mean_squared_error: 9.7958e-04
Epoch 5/20
219/219 [==============================] - 1s 5ms/step - loss: 8.4552e-04 - mean_squared_error: 8.4552e-04
Epoch 6/20
219/219 [==============================] - 1s 6ms/step - loss: 7.8782e-04 - mean_squared_error: 7.8782e-04
Epoch 7/20
219/219 [==============================] - 1s 6ms/step - loss: 7.6227e-04 - mean_squared_error: 7.6227e-04
Epoch 8/20
219/219 [==============================] - 1s 5ms/step - loss: 7.3231e-04 - mean_squared_error: 7.3231e-04
Epoch 9/20
219/219 [==============================] - 1s 5ms/step - loss: 7.7648

In [44]:
# Using same model to predict multiple steps
new_steps = 10
series = generate_time_series(1, n_steps + new_steps)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:n_steps + new_steps]
X = X_new
for step in range(new_steps):
  y_pred_one = model.predict(X[:, step:])[:, np.newaxis, :]
  X = np.concatenate((X, y_pred_one), axis=1)
y_pred = X[:, n_steps:]
print(y_pred)
print(tf.reduce_mean(keras.losses.MSE(Y_new, y_pred)))

1/1 [==============================] - 0s 12ms/step
[[[ 0.22415358]
  [ 0.28099558]
  [ 0.31323424]
  [ 0.32960907]
  [ 0.3208741 ]
  [ 0.29528764]
  [ 0.24252321]
  [ 0.16057444]
  [ 0.07075588]
  [-0.04076669]]]
tf.Tensor(0.0007105941, shape=(), dtype=float32)


In [ ]:
# Guessing 10 at a times
series = generate_time_series(10000, n_steps + new_steps)
X_train, y_train = series[:7000, :n_steps], series[:7000, -new_steps]
X_val, y_val = series[7000:9000, :n_steps], series[7000:9000, -new_steps]
X_test, y_test = series[9000:, :n_steps], series[9000:, -new_steps]

In [ ]:
model = keras.models.Sequential([
  
])